# Setup


In [1]:
%uv pip install mp-api

Note: you may need to restart the kernel to use updated packages.


Using Python 3.11.11 environment at: c:\Users\asrosen\AppData\Local\miniconda3\envs\cms
Audited 1 package in 60ms


To carry out this tutorial, you will need your MP API key. Log onto the Materials Projet, go to your [account dashboard](https://next-gen.materialsproject.org/dashboard), and you will find the API key.


In [ ]:
from mp_api.client import MPRester

api_key = ""  # replace with your own API key


The MP API [documentation](https://docs.materialsproject.org/downloading-data/how-do-i-download-the-materials-project-database) is a useful reference if needed.


# Retrieving data using MPRester

- The `MPRester` contains many convenience functions for getting many common types of data such as crystal structures
- Every material on the Materials Project has a unique `material_id`, which can be readily viewed on the Materials Project website
- To start, we will use some known `material_id` to get data for that material


In [3]:
# Retrieve the crystal structure for a specific material by its Materials Project ID

material_id = "mp-169"  # Example material ID for C

# Fetch structure for the material
with MPRester(api_key=api_key) as mpr:
    structure = mpr.get_structure_by_material_id(material_id)

# Print fetched data
print(structure)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Full Formula (C2)
Reduced Formula: C
abc   :   2.456455   2.456455   3.764525
angles:  71.503370  71.503370  60.002969
pbc   :       True       True       True
Sites (2)
  #  SP           a         b         c    magmom
---  ----  --------  --------  --------  --------
  0  C     0.833356  0.833356  -9.3e-05         0
  1  C     0.166644  0.166644   9.3e-05         0


In [4]:
# Write out structure to CIF to view in VESTA
structure.to(filename=f"{material_id}.cif")


"# generated using pymatgen\ndata_C\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   2.45645465\n_cell_length_b   2.45645465\n_cell_length_c   3.76452542\n_cell_angle_alpha   71.50336991\n_cell_angle_beta   71.50336991\n_cell_angle_gamma   60.00296887\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   C\n_chemical_formula_sum   C2\n_cell_volume   18.30544095\n_cell_formula_units_Z   2\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  C  C0  1  0.83335553  0.83335553  -0.00009304  1\n  C  C1  1  0.16664447  0.16664447  0.00009304  1\n"

In [5]:
# Get some other properties by material ID
with MPRester(api_key=api_key) as mpr:
    rho = mpr.get_charge_density_from_material_id(material_id)

rho.write_file(f"CHGCAR_{material_id}")  # CHGCAR tells VESTA it is an electron density

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving TaskDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

# Using Sub-Resters for Additional Functionality

- The built-in convenience functions of `MPRester` are just a small fraction of the functionality available through the Materials Project REST API
- To access other functions and data as well as to search for materials, you need to use one of the many end points
- Each end point helps retrieve a particular type of data from the Materials Project using one of the REST API endpoints. A full list can be found [here](https://docs.materialsproject.org/downloading-data/using-the-api/getting-started)


- One of the most common end point you may want to use is the `materials/summary` end point, which produces a `SummaryDoc` that contains summary information about materials and their properties
- To see all the available search keyword arguments, "inspect" the `.search` method in VS Code


In [6]:
# Query the summary doc for two materials
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(material_ids=["mp-169", "mp-123"])


Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Print out the first `SummaryDoc`

print(docs[0])

MPDataDoc<SummaryDoc>
builder_meta=EmmetMeta(emmet_version='0.84.7rc1', pymatgen_version='2025.3.10', run_id='32bfb79c-5ce0-41ab-ab69-69ba9fb96205', batch_id='recover-lost-r2scan-may2022', database_version='2025.09.25', build_date=datetime.datetime(2025, 4, 3, 18, 51, 43, 856000, tzinfo=datetime.timezone.utc), license='BY-C'),
nsites=2,
elements=[Element C],
nelements=1,
composition=Composition('C2'),
composition_reduced=Composition('C1'),
formula_pretty='C',
formula_anonymous='A',
chemsys='C',
volume=18.305440952101463,
density=2.179050113817426,
density_atomic=9.152720476050732,
symmetry=SymmetryData(crystal_system=<CrystalSystem.trig: 'Trigonal'>, symbol='R-3m', number=166, point_group='-3m', symprec=0.1, angle_tolerance=5.0, version='2.6.0'),
property_name='summary',
material_id=MPID(mp-169),
deprecated=False,
deprecation_reasons=None,
last_updated=datetime.datetime(2025, 9, 23, 22, 39, 44, 37000, tzinfo=datetime.timezone.utc),
origins=[PropertyOrigin(name='structure', task_id=MPID

In [8]:
# Query the SummaryDoc
# For a list of all the attributes, do `dir(docs[0])` or (better yet) view the SummaryDoc type hint
print(docs[0].formula_pretty)
print(docs[0].density)
print(docs[0].band_gap)


C
2.179050113817426
0.094599999999999


In [9]:
# Now let's do a more complex query without relying on a specific MPID
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(
        elements=["Fe", "O"], exclude_elements=["H"], formation_energy=(-1, 0)
    )

Retrieving SummaryDoc documents:   0%|          | 0/233 [00:00<?, ?it/s]

In [10]:
print(
    f"Found {len(docs)} iron oxides without hydrogen with formation energy between -1 and 0 eV/atom."
)

Found 233 iron oxides without hydrogen with formation energy between -1 and 0 eV/atom.


In [11]:
# Let's look at some of these
print(docs[0].formula_pretty)
print(docs[1].formula_pretty)
print(docs[2].formula_pretty)


AlFe3(SO7)6
AlFe4Hg2O2F
Ba2Ti2Fe2As4O


In [12]:
# We can restrict to a given "chemical space" via the `chemsys` parameter
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(chemsys=["Fe-O"], formation_energy=(-1, 0))

Retrieving SummaryDoc documents:   0%|          | 0/11 [00:00<?, ?it/s]

In [13]:
print(docs[0].formula_pretty)
print(docs[-2].formula_pretty)
print(docs[-1].formula_pretty)

Fe10O11
FeO
FeO3


In [14]:
# Sometimes searches are really data-intensive, in which case you should only return what you need
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(
        elements=["Fe", "O"], fields=["material_id", "structure"]
    )

Retrieving SummaryDoc documents:   0%|          | 0/9182 [00:00<?, ?it/s]

In [15]:
# we stored the .structure attribute
docs[0].structure

Structure Summary
Lattice
    abc : 10.659052064297416 10.45738117176136 9.944885184481619
 angles : 93.7069475994413 87.57439992446837 98.78090990323253
 volume : 1092.637619278804
      A : 10.625443 -0.815076999999999 0.225834999999999
      B : -0.794983 10.421934 -0.328808
      C : 0.186647999999999 -0.317376999999999 9.938067
    pbc : True True True
PeriodicSite: Fe (0.5613, 8.791, 0.8018) [0.115, 0.8557, 0.1064]
PeriodicSite: Fe (2.175, 6.271, 8.134) [0.2383, 0.6457, 0.8344]
PeriodicSite: Fe (10.71, -0.4801, 8.581) [0.9979, 0.05763, 0.8426]
PeriodicSite: Fe (8.174, 9.491, 1.363) [0.84, 0.9809, 0.1505]
PeriodicSite: Fe (-0.06504, 10.07, 3.942) [0.05996, 0.9837, 0.4279]
PeriodicSite: Fe (0.8999, 8.58, 6.394) [0.1369, 0.8543, 0.6685]
PeriodicSite: Fe (9.072, 5.152, 0.7132) [0.895, 0.5665, 0.07017]
PeriodicSite: Fe (8.442, 7.075, 3.004) [0.8455, 0.7543, 0.308]
PeriodicSite: Fe (3.009, 4.586, 1.082) [0.3162, 0.4684, 0.1172]
PeriodicSite: Fe (4.101, 3.76, 7.795) [0.4033, 0.4163, 0.7

# Material ID vs. Task ID

Each "material" on the Materials Project is composed of properties from many individual calculations. Each individual calculation is given a unique Task ID, each of which map to a single parent material ID. We can use the `/materials/tasks` end point to find information about these tasks.


In [16]:
# First, we find the task IDs for each C structure
with MPRester(api_key=api_key) as mpr:
    docs = mpr.materials.summary.search(
        chemsys=["C"], fields=["material_id", "task_ids"]
    )

Retrieving SummaryDoc documents:   0%|          | 0/64 [00:00<?, ?it/s]

In [17]:
for doc in docs:
    print(f"{doc.material_id}: {doc.task_ids}")

mp-1008374: [MPID(mp-2371415), MPID(mp-2371375), MPID(mp-2329075), MPID(mp-1065565), MPID(mp-2681477), MPID(mp-2371512), MPID(mp-2371640), MPID(mp-1065534), MPID(mp-1524876), MPID(mp-2371285), MPID(mp-1065604), MPID(mp-2371403), MPID(mp-2371425), MPID(mp-1434321), MPID(mp-2681531), MPID(mp-2681599), MPID(mp-2371373), MPID(mp-1791553), MPID(mp-2173805), MPID(mp-2371261), MPID(mp-2371621), MPID(mp-1612320), MPID(mp-1065548), MPID(mp-2371557), MPID(mp-1525297), MPID(mp-1008374), MPID(mp-2371321), MPID(mp-2371649), MPID(mp-2371532), MPID(mp-2371620), MPID(mp-1766221), MPID(mp-2371359)]
mp-1008395: [MPID(mp-2681471), MPID(mp-2690008), MPID(mp-2681570), MPID(mp-1067153), MPID(mp-1067136), MPID(mp-2371572), MPID(mp-1063853), MPID(mp-1067130), MPID(mp-2371363), MPID(mp-1432023), MPID(mp-1008395), MPID(mp-2371386), MPID(mp-2371619), MPID(mp-2371516), MPID(mp-1063861), MPID(mp-1008503), MPID(mp-2371326), MPID(mp-2371398), MPID(mp-2074613), MPID(mp-1008512), MPID(mp-1675200), MPID(mp-1794715), MP

In [18]:
# Let's search all the tasks for the first material we found
with MPRester(api_key=api_key) as mpr:
    task_docs = mpr.materials.tasks.search(task_ids=docs[0].task_ids)

Retrieving TaskDoc documents:   0%|          | 0/32 [00:00<?, ?it/s]

In [19]:
# The first task doc
task_docs[0]

MPDataDoc<TaskDoc>(
nsites=4,
elements=[Element C],
nelements=1,
composition=Composition('C4'),
composition_reduced=Composition('C1'),
formula_pretty='C',
formula_anonymous='A',
chemsys='C',
volume=26.53839721596952,
density=3.006095116431695,
density_atomic=6.63459930399238,
symmetry=SymmetryData(crystal_system=<CrystalSystem.ortho: 'Orthorhombic'>, symbol='Cmmm', number=65, point_group='mmm', symprec=0.1, angle_tolerance=None, version='2.5.0'),
state=<TaskState.SUCCESS: 'successful'>,
calcs_reversed=[Calculation(dir_name='/global/projecta/projectdirs/matgen/garden/block_2016-09-14-21-21-57-007032/launcher_2016-09-16-12-55-23-067133', vasp_version='5.2.12', has_vasp_completed=True, input=CalculationInput(incar={'ISTART': 1, 'PREC': 'accurate', 'ALGO': 'Fast', 'ISPIN': 2, 'ICHARG': 1, 'NELM': 100, 'IBRION': 2, 'EDIFF': 0.0002, 'EDIFFG': -0.05, 'NSW': 99, 'ISIF': 3, 'ENCUT': 520.0, 'MAGMOM': [0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001], 'LREAL': 'Auto

In [20]:
# We can even get the input and output parameters for full reproducibility
task_docs[0].input

InputDoc(incar={'PREC': 'accurate', 'ALGO': 'Fast', 'ISPIN': 2, 'ICHARG': 1, 'NELM': 100, 'IBRION': 2, 'EDIFF': 0.0002, 'NSW': 99, 'ISIF': 3, 'ENCUT': 520.0, 'MAGMOM': [0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001], 'LREAL': 'Auto', 'ISMEAR': -5, 'SIGMA': 0.05, 'LWAVE': False, 'LORBIT': False}, kpoints=None, nkpoints=None, potcar=None, potcar_spec=[PotcarSpec(titel='PAW_PBE C 08Apr2002', hash='c0a8167dbb174fe492a3db7f5006c0f8', summary_stats=None)], potcar_type=None, parameters={'SYSTEM': 'unknown system', 'LCOMPAT': False, 'PREC': 'accura', 'ENMAX': 520.0, 'ENAUG': 644.873, 'EDIFF': 0.0002, 'IALGO': 68, 'IWAVPR': 11, 'NBANDS': 14, 'NELECT': 16.0, 'TURBO': 0, 'IRESTART': 0, 'NREBOOT': 0, 'NMIN': 0, 'EREF': 0.0, 'ISMEAR': -5, 'SIGMA': 0.05, 'KSPACING': 0.5, 'KGAMMA': True, 'LREAL': True, 'ROPT': [-0.00025], 'LMAXPAW': -100, 'LMAXMIX': 2, 'NLSPLINE': False, 'ISTART': 0, 'ICHARG': 1, 'INIWAV': 1, 'ISPIN': 2, 'LNONCOLLINEAR': False, 'MAGMOM': [0.6000000000

In [21]:
task_docs[0].output

OutputDoc(structure=Structure Summary
Lattice
    abc : 2.70156816 2.51200063 4.137281909952495
 angles : 90.0 70.94408672694027 90.0
 volume : 26.53839721596952
      A : 2.70156816 0.0 0.0
      B : 0.0 2.51200063 0.0
      C : 1.35078408 0.0 3.91056057
    pbc : True True True
PeriodicSite: C (2.702, 1.256, 1.481) [0.8106, 0.5, 0.3788]
PeriodicSite: C (2.702, 0.0, 0.6711) [0.9142, 0.0, 0.1716]
PeriodicSite: C (1.351, 1.256, 2.429) [0.1895, 0.5, 0.6211]
PeriodicSite: C (1.351, 0.0, 3.239) [0.0859, 0.0, 0.8282], density=3.006095068277374, energy=-35.15284593, forces=None, stress=None, energy_per_atom=-8.7882114825, bandgap=0.0)

In [22]:
with MPRester(api_key=api_key) as mpr:
    thermo_docs = mpr.materials.thermo.search(formula="PbN6")

Retrieving ThermoDoc documents:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
print(thermo_docs[0].formula_pretty)

PbN6


In [24]:
thermo_docs[0].decomposes_to

[DecompositionProduct(material_id=MPID(mp-25), formula='N8', amount=0.8571428571428571),
 DecompositionProduct(material_id=MPID(mp-20483), formula='Pb1', amount=0.14285714285714202)]

In [25]:
# N8? What?
with MPRester(api_key=api_key) as mpr:
    n8_doc = mpr.get_structure_by_material_id("mp-25")
n8_doc.to(filename="N8.cif")  # view in VESTA or MatterViz; oh, it's just N2!

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

"# generated using pymatgen\ndata_N2\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   5.56051766\n_cell_length_b   5.56051766\n_cell_length_c   5.56051766\n_cell_angle_alpha   90.00000000\n_cell_angle_beta   90.00000000\n_cell_angle_gamma   90.00000000\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   N2\n_chemical_formula_sum   N8\n_cell_volume   171.92762867\n_cell_formula_units_Z   4\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  N  N0  1  0.05739733  0.05739733  0.05739733  1\n  N  N1  1  0.44260267  0.94260267  0.55739733  1\n  N  N2  1  0.55739733  0.44260267  0.94260267  1\n  N  N3  1  0.94260267  0.55739733  0.44260267  1\n  N  N4  1  0.94260267  0.94260267  0.94260267  1\n  N  N5  1  0.55739733  0.05739733  0.44260267  1\n  N  N6  1  0.44260267  0

In [26]:
n8_doc.reduced_formula

'N2'

# ToDo


## Exploration


Now it's your turn. Go to the Materials Project website and pick any material that makes you happy. Take a note of the Material ID. Then, using the MP API's `MPRester.materials.summary.search`, answer the following question:

Using the `.search` method, put together a query that would return your material in the resulting list. The list that is returned is of type `list[SummaryDoc]`. Confirm that you indeed returned the desired structure by iterating through the list and accessing the `.material_id` attribute. Try to make your query as specific as possible so that you are returning only the material(s) of interest.


In [ ]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(
        # input keyword arguments and values here
    )

c:\Users\asrosen\AppData\Local\miniconda3\envs\cms\Lib\site-packages\mp_api\client\core\client.py:1306: UserWarning: Omitting a query also includes deprecated documents in the results. Make sure to post-filter them out.
  warnings.warn(


Retrieving SummaryDoc documents:   0%|          | 0/210579 [00:00<?, ?it/s]

c:\Users\asrosen\AppData\Local\miniconda3\envs\cms\Lib\site-packages\pymatgen\core\structure.py:2917: UserWarning: Not all sites have property charge. Missing values are set to None.
  return cls.from_sites(sites, charge=charge, properties=dct.get("properties"))
c:\Users\asrosen\AppData\Local\miniconda3\envs\cms\Lib\site-packages\pymatgen\core\structure.py:2917: UserWarning: Not all sites have property velocities. Missing values are set to None.
  return cls.from_sites(sites, charge=charge, properties=dct.get("properties"))
c:\Users\asrosen\AppData\Local\miniconda3\envs\cms\Lib\site-packages\pymatgen\core\structure.py:2917: UserWarning: Not all sites have property selective_dynamics. Missing values are set to None.
  return cls.from_sites(sites, charge=charge, properties=dct.get("properties"))
c:\Users\asrosen\AppData\Local\miniconda3\envs\cms\Lib\site-packages\pymatgen\core\structure.py:2917: UserWarning: Not all sites have property coordination_no. Missing values are set to None.
  r

In [ ]:
# Write a simple code to confirm your desired material has been returned.


## Complex Query


Here is a harder question if you are feeling ambitious. How many materials on the Materials Project are experimentally synthesized (i.e. not theoretical), are **ternary** compounds (i.e. materials with 3 unique elements), and contain **at least Ca and Al**? Bonus: Which one has the lowest density?


In [ ]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(
        # input keyword arguments and values here
    )

In [ ]:
# Write a simple code to determine how many entries are returned

# ToDo Answer Key


## Exploration


The answer here will depend on your material. I chose CsAu. It has Material ID `mp-2667`.


In [27]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(num_elements=2, elements=["Cs", "Au"])

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
for material in materials:
    print(material.material_id)

mp-2667


## Complex Query


We start by making our multi-parameter query. If you aren't sure what keywords are available, you should inspect the definition of the `.search` method or read the MP API documentation.


In [29]:
with MPRester(api_key=api_key) as mpr:
    materials = mpr.materials.summary.search(
        theoretical=False, num_elements=3, elements=["Ca", "Al"]
    )

Retrieving SummaryDoc documents:   0%|          | 0/66 [00:00<?, ?it/s]

Then we can get the length of the returned `list[Structure]` to see how many entries there are:


In [30]:
materials[0].material_id
print(f"The number of entries is {len(materials)}")

The number of entries is 66


Alright, now how about the lowest density material? We can iterate through the `list[SummaryDoc]` and find the one with the lowest `.density` attribute.


In [31]:
import numpy as np

density = np.inf
for material in materials:
    if material.density < density:
        lowest_density_material = material
        density = material.density

print(
    f"The lowest density one is {lowest_density_material.material_id} with density {lowest_density_material.density} g/cm^3"
)

The lowest density one is mp-1198688 with density 1.159853032513954 g/cm^3


If you wanted to do the above code more efficiently:


In [32]:
idx = np.argmin([material.density for material in materials])
lowest_density_material = materials[idx]
print(
    f"The lowest density one is {lowest_density_material.material_id} with density {lowest_density_material.density} g/cm^3"
)

The lowest density one is mp-1198688 with density 1.159853032513954 g/cm^3


Finally, we look up the material on the Materials Project to make sure it looks reasonable: https://next-gen.materialsproject.org/materials/mp-1198688?material_ids=mp-1198688


# Being faster and more data-efficient with Materials Project


Knowing how to be data-efficient when using the REST API is good for several reasons. It is helpful for you because it results in much faster results and lower data/memory usage on your side when conducting analyses. It is also much better for the Materials Project because it avoids unnecessary data transfer costs. There are several things you can do to make your queries faster and more data-efficient:

- As already covered, restrict the data returned to the specific fields of interest, to the extent possible:

```python
with MPRester("your_api_key_here") as mpr:
    docs = mpr.materials.summary.search(fields=["material_id", "volume", "elements"])
```

- If you are just exploring / testing queries and don't want to wait for thousands of results to be retrieved, use `num_chunks=1` and `chunk_size=10` parameters when calling `search()` to limit to 10 example results. This works for all searches with all Resters and avoids unnecessary calls:

```python
with MPRester("your_api_key_here") as mpr:
    mpr.summary.search(band_gap=[0,10], num_chunks=1, chunk_size=10)
```

- If you need to get data for many materials, pass the `materials_ids` as a list. This minimizes the number of calls to the API (i.e. don't call `search()` thousands of times!):

```python
with MPRester("your_api_key_here") as mpr:
    docs = mpr.materials.summary.search(material_ids=["mp-149", "mp-13", "mp-22526"])
```

- For more tips, see https://docs.materialsproject.org/downloading-data/using-the-api/tips-for-large-downloads
